In [6]:
from dotenv import load_dotenv
import os

load_dotenv()

True

In [5]:
from langchain.document_loaders import PyPDFLoader
loader = PyPDFLoader("../data/Constitución Española/Constitución española 3-7.pdf")
documents = loader.load()
for document in documents:
    document.metadata['filename'] = document.metadata['source']

from langchain_text_splitters import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=450,
    chunk_overlap=100,
    length_function=len,
    is_separator_regex=False,
)

texts = text_splitter.split_documents(documents)

In [8]:
from langchain.embeddings import AzureOpenAIEmbeddings
from langchain.vectorstores import Chroma
import os

embedding_function = AzureOpenAIEmbeddings(
    azure_deployment=os.environ["AZURE_ADA2_DEPLOYMENT"],
    openai_api_version=os.environ["OPENAI_API_VERSION"],
)

# load docs into Chroma
vector_db = Chroma.from_documents(texts, embedding_function, persist_directory='../vector_stores/chroma_openai_ada2')

# Helpful to force a save
vector_db.persist()

# get db connection
vector_db_connection = Chroma(persist_directory='../vector_stores/chroma_openai_ada2', embedding_function=embedding_function)

In [9]:
retriever = vector_db_connection.as_retriever(search_kwargs={"k": 3})

In [11]:
from langchain_openai import AzureChatOpenAI
from langchain.chains import RetrievalQA

# create a retrieval qa chain using llm
llm = AzureChatOpenAI(
    openai_api_version=os.environ["OPENAI_API_VERSION"],
    azure_deployment=os.environ["AZURE_GPT3TURBO_DEPLOYMENT"],
)
qa = RetrievalQA.from_chain_type(llm=llm,
                                 chain_type="stuff",
                                 retriever=retriever,
                                 verbose=True)

In [13]:
result = qa.invoke("How is the right to choose one's residence and to move freely within the national territory guaranteed in the Spanish Constitution?")



> Entering new RetrievalQA chain...

> Finished chain.


In [20]:
retriever.get_relevant_documents("How is the right to choose one's residence and to move freely within the national territory guaranteed in the Spanish Constitution?")

[Document(page_content='y familiar de los ciudadanos y el pleno ejercicio de sus derechos.\nArtículo 19.  \nLos españoles tienen derecho a elegir libremente su residencia y a circular por el \nterritorio nacional.\nAsimismo, tienen derecho a entrar y salir libremente de España en los términos que la \nley establezca. Este derecho no podrá ser limitado por motivos políticos o ideológicos.\nArtículo 20.  \n1. Se reconocen y protegen los derechos:', metadata={'filename': '../data/Constitución Española/Constitución española 3-7.pdf', 'page': 3, 'source': '../data/Constitución Española/Constitución española 3-7.pdf'}),
 Document(page_content='Artículo 13.  \n1. Los extranjeros gozarán en España de las libertades públicas que garantiza el \npresente Título en los términos que establezcan los tratados y la ley.\n2. Solamente los españoles serán titulares de los derechos reconocidos en el artículo 23, \nsalvo lo que, atendiendo a criterios de reciprocidad, pueda establecerse por tratado o ley 